# Quantifying off-sample performance

Having established some context through the detailed [warm-up example from the previous section](./warmup.ipynb), recall that the key topic of this entire article is _designing learning algorithms to achieve a desirable loss distribution._ In the previous example, we started with a well-known dataset including outliers, examined the performance of some standard learning algorithms, and based on this performance we decided on the properties of the loss distribution that we would consider "desirable" given the learning task at hand. We then used this as a guide to design an alternative procedure (the median loss minimizer) which, leaving aside issues of scalability, behaved as we would hope and outperformed the more standard procedures. 

In this section, the primary focus is on the means for actually quantifying different properties of the underlying loss distribution. While it was natural for us to use the loss median in the previous example, more generally there will be other properties we would like to emphasize (or ignore), and the goal of this section is to assemble a toolkit to quantify such properties.

## Basic formulation


To get started, we introduce some formal notation to faciliate our exposition. In machine learning, much attention is paid to the precise form of the loss function $\ell(h;x,y)$. This was emphasized by the OLS/LAD comparison in the previous section. For our purposes, however, the exact nature of the loss function is not important (yet). Furthermore, there is no need to restrict ourselves to a "supervised learning" task with an input-output pair $(x,y)$, either. Our interest is in the probability distribution of the loss over the random draw of the data that the loss depends on.

Making all this a bit more explicit, let $Z \sim \mu$ be a random data point with distribution $\mu$, and let us use $\mathrm{L}(h) = \ell(h;Z)$ to denote the random loss of interest, where the loss function $\ell$ can in principle be any real-valued function. To keep ourselves conceptually grounded, we will refer to $\ell(\cdot;\cdot)$ as the __base loss function__ and $\mathrm{L}(h)$ as simply the __base loss__. This general-purpose notation emphasizes the fact that our basic object of interest is the distribution of $\mathrm{L}(h)$.

While the loss distribution is of critical importance, it is quite unwieldy as an indicator of learning algorithm performance. The task of choosing $h$ such that $\mathrm{L}(h)$ is "optimal" is of course an ill-posed problem, since in general, there are countless different ways in which we could measure the optimality of a probability distribution. As such, we must transform the random loss $\mathrm{L}$ into a single real value that can be used to make statements of "optimality" precise and meaningful. As a generic notation, we will denote this transform by

\begin{align}
\mathrm{L} \mapsto \mathrm{R}(\mathrm{L}) \in \mathbb{R}.
\end{align}

When we want to emphasize the role of $h$, we will overload our notation and write $\mathrm{R}(h)$ instead of $\mathrm{R}(\mathrm{L}(h))$. We will refer to $\mathrm{R}(\mathrm{L})$ as the __risk__ incurred by $\mathrm{L}$, with $\mathrm{R}(\cdot)$ being the __risk function__. Without question, the _de facto_ standard setting in modern machine learning is to focus on <u>average</u> performance, i.e., to quantify optimality as

\begin{align}
\mathrm{R}(h) = \mathbf{E}_{\mu} \mathrm{L}(h).
\end{align}

The expected loss to be incurred by $h$ is a perfectly natural way to define optimal performance. Indeed, both OLS and LAD discussed in the previous example are defined to minimize the (empirical) average loss. However, as the previous example also showed us, there may be clear advantages to considering a different definition of optimality (e.g., the median loss).

Moving forward, we will consider different concrete forms for the risk $\mathrm{L} \mapsto \mathrm{R}(\mathrm{L})$ to take, and give some empirical examples which highlight the different properties of the loss distribution that can be picked up by different risk function classes. In the end, however, there are always tradeoffs. Recalling our statement from the previous section:

> Unfortunately, the reality of machine learning in the wild is that we only have limited data and prior knowledge at training time, and thus _"make all losses minimal!"_ is a totally unrealistic goal. Not all losses can be made minimal. Tradeoffs are inevitable. In order to responsibly design effective and transparent learning systems, we need the _vocabulary_ to make these tradeoffs explicit.

Designing a risk function forces us to make decisions about what to prioritize, and what to ignore.

## Generalizing the notion of location

Since the expected value of a random value is widely considered to be the canonical location parameter, we shall start by generalizing the notion of the "location" of a probability distribution. As a motivating fact, first recall that if $\mathrm{L}$ has finite variance, the its expected value satisfies

\begin{align}
\mathbf{E}_{\mu}(\mathrm{L}-\mathbf{E}_{\mu}\mathrm{L})^{2} \leq \mathbf{E}_{\mu} (\mathrm{L}-\theta)^{2}
\end{align}

for any choice of $\theta \in \mathbb{R}$. The mean is the location about which the distribution is most sharply concentrated, in terms of the expected squared deviation. That said, in general, if we change the way in which we measure deviations, the optimal location will change. As such, it is natural to define a general class of location parameters as a solution set

\begin{align}
M_{\rho}(\mathrm{L}) = \arg\min_{\theta \in \mathbb{R}} \, \mathbf{E}_{\mu} \rho(\mathrm{L}-\theta)
\end{align}

where the function $\rho: \mathbb{R} \to \mathbb{R}_{+}$ is in charge of measuring deviations. The elements of the solution set $M_{\rho}(\mathrm{L})$ are called _M-parameters_ of the distribution of $\mathrm{L}$ \[Kol97\]. In the special case of $\rho(\cdot) = (\cdot)^{2}$ and $\rho(\cdot) = \lvert \cdot \rvert$, we can guarantee that $M_{\rho}(\mathrm{L})$ includes the mean and median of $\mathrm{L}$ respectively. One can easily design $\rho$ to modulate between these two extremes; as an example from the literature \[Hol21\], let us set

\begin{align}
\rho(x) = \eta \widetilde{\rho}(x/\sigma), \text{ where } \widetilde{\rho}(x) = x\,\mathrm{atan}(x) + \frac{\log(1+x^{2})}{2}
\end{align}

with parameters $\sigma > 0$ and $\eta > 0$ used for re-scaling. The function $\widetilde{\rho}$ used here may appear a bit strange, but it is actually quite simple and useful in that it behaves approximately quadratically around zero, and it is both $\pi/2$-Lipschitz and strictly convex on $\mathbb{R}$. For reference, in the following figure we plot the graphs of $\widetilde{\rho}$ (solid line), and its first two derivatives (dashed, dot-dashed respectively).

![Dispersion fn (mrisk)](img/basics_dev_mrisk.png)

Let us use this class of M-parameters to modulate between the mean and median of a probability distribution. As mentioned in \[Hol21, Rmk. 2\], we have that

\begin{align}
2\sigma^{2}\widetilde{\rho}(x/\sigma) \to x^{2} \text{ as } \sigma \to \infty, \text{ and } 2(\sigma/\pi)\widetilde{\rho}(x/\sigma) \to \lvert x \rvert \text{ as } \sigma \to 0_{+}.
\end{align}

This gives us a precise re-scaling strategy (i.e., precise settings of $\eta$ and $\sigma$) such that the resulting M-parameter approaches the mean as $\sigma$ gets large, and the median as $\sigma$ gets close to zero. For all practical purposes, though, we do not need to be quite so precise. Let us see a numerical example of this in action. We simulate the distribution of $\mathrm{L}$ using some well-known parametric distributions, and for ease of numerical comparison, we shift the distributions such that $\mathbf{E}_{\mu}\mathrm{L}=0$. We then take a large iid sample from this distribution, and using the `minimize_scalar` function [included in SciPy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize_scalar.html) we seek a minimizer of the map $\theta \mapsto \mathbf{E}_{\mu}\rho(\theta-\mathrm{L})$, replacing $\mu$ with the empirical distribution. As a simple first example, we look at the case of a Bernoulli distribution (this might arise when using the zero-one loss function for classification).

![Risk visuals (M-location, Bernoulli)](img/basics_riskvis_mean2med_bernoulli.png)

The solid black line denotes the optimal $\theta$ value found by the minimizer, for a variety of choices of $\sigma$. The dashed black line is the value of the objective function achieved at this minimizer; we shade in the difference between these two values to emphasize the degree of dispersion. Mean and median values are given by blue and red horizontal rules. The difference between the "mean scale" and "median scale" settings is simply that of re-scaling with $\eta = 2\sigma^{2}\widetilde{\rho}(x/\sigma)$ and $\eta = 2(\sigma/\pi)$ respectively. The same trends can of course be observed on continuous distributions as well:

![Risk visuals (M-location, Beta)](img/basics_riskvis_mean2med_beta.png)
![Risk visuals (M-location, Pareto)](img/basics_riskvis_mean2med_pareto.png)

Note how the mean can be pulled both above and below the median depending on the direction of the distribution tails. Regardless of direction, this $\sigma$-based modulation works as expected. In the special case where the distribution of $\mathrm{L}$ is symmetric, the M-locations under this symmetric $\rho$ all coincide. Take for example the case of a Normal distribution.

![Risk visuals (M-location, Normal)](img/basics_riskvis_mean2med_normal.png)

In any case, however, this class of M-locations clearly gives us significant flexibility in terms of "sensitivity to tails" when deciding how we want define the location of a loss distribution. Using the strict convexity of the function $\widetilde{\rho}(\cdot)$ just introduced, the solution set $M_{\rho}(\mathrm{L})$ contains a single element, and it is thus intuitive to design a risk function family of the form

\begin{align}
\mathrm{R}_{\text{M}}(h) = M_{\rho}(\mathrm{L}(h)).
\end{align}

Minimizers of the empirical variant of this objective can be shown to enjoy some nice robustness properties \[BJL15\] in principle, but note that from a computational viewpoint, this is a "bi-level program" in that the objective function in $h$ is itself the _solution_ of a separate optimization problem (the M-parameter characterization). Solving such problems directly introduces a significant computational overhead, limiting the usefulness of this approach for larger-scale machine learning tasks. Moving forward, we go beyond this class of location parameters and consider some notions of location which are more computationally congenial.

## Flexible worst-case sensitivity

In the previous example, we saw how one can modulate between the median and mean of the underlying loss distribution. In terms of tradeoffs, the closer we are to the median, the more weight we put on ensuring that (given a sample of losses) the _majority_ of losses, albeit not all losses, are small. As a result, less weight is placed on penalizing worst-case examples. In contrast to this, most of the best-known risk functions proposed as alternatives to the expected loss are naturally interpreted as placing more weight on the worst-case than the mean does, with less weight placed on the majority. Here we introduce and examine some well-known examples.

### Conditional value-at-risk

Perhaps the simplest modification to the mean loss is to consider a _conditional_ expectation, conditioned on some kind of "worst case" event. One of the most natural definitions of such an event is the case in which the loss exceeds a certain threshold, for example, a certain quantile. Making this explicit, let us denote the $\beta$-level quantile of $\mathrm{L}(h)$ for any $0 < \beta < 1$ by

\begin{align}
\mathrm{Q}_{\beta}(h) = \inf\{x \in \mathbb{R}: \mathbf{P}\{\mathrm{L}(h) \leq x\} \geq \beta\}
\end{align}

and then consider the conditional expectation

\begin{align}
\mathrm{R}_{\text{CVaR}}(h) = \mathbf{E}_{\mu}[\mathrm{L}(h) \,\vert\, \mathrm{L}(h) \geq \mathrm{Q}_{\beta}(h)].
\end{align}

This risk function is well-known as conditional value-at-risk (CVaR) \[RU00\].


### Tilted risk

Instead of conditioning on the event of exceeding a certain quantile, one could of course just target a particular quantile $h \mapsto \mathrm{Q}_{\beta}(h)$ directly. However, as we mentioned [in the previous section](./warmup.ipynb) in the special case of $\beta = 1/2$, quantiles of random losses need not inherent convexity or smoothness properties of the base loss functions underlying them. A smooth approximation of both low- and high-level quantiles can be obtained using

\begin{align}
\mathrm{R}_{\text{tilt}}(h) = \frac{1}{\gamma}\log\left(\mathbf{E}_{\mu}\mathrm{e}^{\gamma\mathrm{L}(h)}\right)
\end{align}

and taking the absolute value of parameter $\gamma \neq 0$ sufficiently large. A larger value of $\gamma$ gives a sharper estimate, but the objective function becomes less smooth. This class of risk functions has been called "tilted risk" in the machine learning literature \[LBSS21\].


### Optimized certainty equivalent risk

The two preceding risk classes are closely related to another, much larger class of risk functions. Let $\phi: \mathbb{R} \to \mathbb{R}$ be a monotonic non-decreasing, closed, convex function which satisfies both $\phi(0) = 0$ and $1 \in \partial\phi(0)$. That is, it is non-positive on the negative half-line, positive on the positive half-line, and must either be differentiable at 0 with a slope of 1, or include the slope of 1 in its sub-differential. Given such a function $\phi$, the optimized certainty equivalent (OCE) risk is defined by

\begin{align}
\mathrm{R}_{\text{OCE}}(h) = \inf_{\theta \in \mathbb{R}} \, \left[ \theta + \mathbf{E}_{\mu}\phi(\mathrm{L}(h)-\theta) \right].
\end{align}

The class of OCE risks includes many special cases \[LPS20\]. Trivially we obtain the expected loss by setting $\phi(x) = x$. We can obtain any $\beta$-level CVaR introduced earlier by setting $\phi(x) = \max\{0,x\}/(1-\beta)$, and can recover the special case of tilted risk with $\gamma > 0$ by setting $\phi(x) = (\mathrm{e}^{\gamma x}-1)/\gamma$. Note that if we define $\varphi(x) = \phi(x) - x$, we can trivially re-write any OCE risk as

\begin{align}
\mathrm{R}_{\text{OCE}}(h) = \mathbf{E}_{\mu}\mathrm{L}(h) + \inf_{\theta \in \mathbb{R}} \, \mathbf{E}_{\mu}\varphi(\mathrm{L}(h)-\theta),
\end{align}

and since by the assumptions made on $\phi$ we have $\varphi(x) \geq 0$, it is evident that all OCE risks are at least as sensitive to extreme events as the mean is. This expression also allows for the interpretation of OCE risks as being a sum of "location" and "deviation" parameters of the underlying distribution; we will return to this notion shortly. Since $\phi(\cdot)$ is assumed to be convex _and_ monotonic, whenever the base loss function is convex in $h$, this property is carried over over to the OCE risk. Furthermore, unlike the M-locations, the risk is characterized as the minimum value (rather than the minimizer itself), meaning that it is quite straightforward to optimize jointly in $(h,\theta) \in \mathcal{H} \times \mathbb{R}$.


### Distributionally robust optimization risk

Another important set of risk functions considers a "robust regularization" of the expected loss, by accounting for potential shift in the underlying data distribution. Written succinctly, instead of just paying attention to a single loss $\mathrm{L}(h) = \ell(h;Z)$ with $Z \sim \mu$, one considers the worst case out of a family of random losses $\mathcal{L}$, i.e., a risk function of the general form

\begin{align}
\mathrm{R}(h) = \sup\left\{\mathbf{E}[\mathrm{L}(h)] : \mathrm{L} \in \mathcal{L}\right\}.
\end{align}

The exact nature of this risk function of course depends entirely on how the set $\mathcal{L}$ is defined. A typical example is a ball of radius $a \geq 0$ set around the initial data distribution $\mu$, i.e., essentially we have $\mathcal{L} = \{\ell(\cdot;Z): Z \sim \nu, \lVert \nu - \mu \rVert \leq a\}$, where the norm condition "$\lVert \nu - \mu \rVert \leq a$" is often weakened to be measured with some divergence function (not necessarily a proper metric). One of the most well-studied examples is an $a$-radius ball measured using the $\chi^{2}$ divergence \[ZDKR21\], which yields a risk function with a form quite reminiscent of the OCE risks we saw earlier:

\begin{align}
\mathrm{R}_{\chi^{2}}(h) = \inf_{\theta \in \mathbb{R}} \, \left[ \theta + \sqrt{(1+2a)\mathbf{E}_{\mu}(\mathrm{L}(h)-\theta)_{+}^{2}} \right]
\end{align}

for $a \geq 0$, noting that $(\cdot)_{+} = \max\{\cdot,0\}$. Minimizing risks of this form encourage a certain degree of robustness to changes in the underlying distribution, and thus we call this risk the $\chi^{2}$-DRO risk, where DRO stands for distributionally robust optimization. While strictly speaking this is not an OCE risk, it is quite similar in that it is always at least as sensitive to extreme values as the mean is, though that sensitivity is defined in a distinct way. It also shares the convenient property that the convexity of the loss function is carried over to the $\chi^{2}$-DRO risk, and solving for $(h,\theta)$ simultaneous is practical.


### A numerical comparison

As we did earlier with the M-location example, let us visualize the three risk function classes considered here on some simulated data. Since we want to visualize _classes_ and not individual risks, we consider the risk values returned by CVaR over a range of $\beta$ choices, tilted risk over a range of $\gamma$ choices, and $\chi^{2}$-DRO risk over a range of $a$ choices. The following figure is analogous to that obtained for the M-locations earlier.

![Risk visuals (worst case guys, Bernoulli)](img/basics_riskvis_worstcase_bernoulli.png)

Some attention is required when parsing this figure. First of all, as with the M-location case, risk computations are done by minimizing an objective over $\theta \in \mathbb{R}$; the optimal value of $\theta$ found is denoted by a solid black line, and the objective function value is denoted by a dashed black line. This is identical to the M-location case, but note that the _risk definitions_ are in terms of the objective function value, rather than the optimal $\theta$. This point is different from the M-location example. As such, risk values are denoted by dashed black lines here. Everything else is the same. Another point is regarding the tilted risk. For $\gamma > 0$ it is an OCE risk, and some basic calculus shows us that the optimal $\theta$ is in fact equal to the minimal objective function value. For $\gamma < 0$, it is not an OCE risk, but it can still be computed analytically, which we have done. As such, there is only one line for the tilted risk in all cases. Finally, the horizontal axis for $\chi^{2}$-DRO risk is $\widetilde{a}$, not $a$. We have used the common re-parametrization of $a = ((1-\widetilde{a})^{-1}-1)^{2}/2$ for numerical convenience.

For reference, results for the other data distributions are given below.

![Risk visuals (worst case guys, Beta)](img/basics_riskvis_worstcase_beta.png)
![Risk visuals (worst case guys, Pareto)](img/basics_riskvis_worstcase_pareto.png)
![Risk visuals (worst case guys, Normal)](img/basics_riskvis_worstcase_normal.png)

Note that in all cases, as the worst-case sensitivity is reduced ($\beta \to 0$, $\gamma \to 0$, $a \to 0$), the risk value returned by each class approachs the mean, as we would anticipate. As the sensitivity levels are raised, it is apparent that these classes tend to be much more sensitive to tails on the upside than the M-locations we looked at earlier, and indeed, the notion of "location" can shift quite far from the mean and median. Closely related to this point, note that regardless of the symmetry of the data, these risk classes can be readily taken towards extreme values.

## Additional flexibility based on dispersion

Let us briefly recapitulate the main points we have covered thus far. M-locations can be used as a flexible representation of distribution location (e.g., modulating between the mean and median), but computational issues mean that they cannot be a workhorse of large-scale machine learning, but rather just play a supporting role. OCE risks and DRO risks are much more computationally convenient, but their expressive power is limited in that all they are designed to do is add a worst-case penalty to the expected loss. In addition, they tend to be highly _asymmetric_ in that this "worst-case penalty" only effectively measures deviations in the upward direction (i.e., extremely large losses beyond a certain threshold). To see this, first recall that we were able to write any OCE risk as

\begin{align}
\mathrm{R}_{\text{OCE}}(h) = \mathbf{E}_{\mu}\mathrm{L}(h) + \inf_{\theta \in \mathbb{R}} \, \mathbf{E}_{\mu}\varphi(\mathrm{L}(h)-\theta)
\end{align}

where $\varphi(x) = \phi(x) - x$ is essentially tasked with measuring deviations that will be used as an extra penalty on top of the mean. Since the OCE class demands that $\phi(\cdot)$ grow slowly (or not at all) on the negative half-line, the resulting deviations measured by $\varphi(\cdot)$ tend to be skewed toward the upside (i.e., losses much larger than the threshold $\theta$), and are insensitive to deviations on the downside (e.g., in the case of losses that are unbounded below, negative rewards, etc.). Plus, the "location" one can capture is always as least as large as the mean. As a systematic and flexible way to address these issues and completement the existing risk classes, a recent line of work \[Hol22\] has considered replacing the OCE-style $\phi(\cdot)$ with a function $\rho_{\alpha}(\cdot)$ from the following class:

\begin{align}
\rho_{\alpha}(x) = 
\begin{cases}
x^{2}/2, & \text{if } \alpha = 2\\
\log\left(1 + x^{2}/2\right), & \text{if } \alpha = 0\\
1 - \exp\left(-x^{2}/2\right), & \text{if } \alpha = -\infty\\
\frac{\lvert \alpha-2 \rvert}{\alpha}\left(\left(1 + \frac{x^{2}}{\lvert \alpha-2 \rvert}\right)^{\alpha/2} - 1\right), & \text{otherwise}.
\end{cases}
\end{align}

For ease of reference we call this the "Barron class" of functions for measuring deviations, since it was used by J.T. Barron with great effect in computer vision applications for _loss function_ design (quite different from our usage here) \[Bar19\]. While the definition looks complicated, the basic traits can be summarized succinctly as follows.

- Regardless of the parameter $\alpha$, the function $\rho_{\alpha}$ is essentially quadratic around zero.
- At the extreme end of $\alpha = 2$, $\rho_{\alpha}$ is a quadratic function.
- For $\alpha \geq 1$, $\rho_{\alpha}$ is convex. Moving from $\alpha = 2$ to $\alpha = 1$, the growth slows from quadratic to linear.
- For $\alpha < 1$, inflection points appear and the function becomes concave far from zero.
- Moving from $\alpha = 1$ to $\alpha = 0$, the growth slows from linear to logarithmic.
- For $\alpha < 0$, the function is bounded.

It is instructive to visually compare these classes of deviation functions.

![Compare dispersion fns](img/basics_dispersions.png)

|Barron|CVaR|Tilted|$\chi^{2}$-DRO|
|:-:|:-:|:-:|:-:|
|![Barron](img/basics_colorbar_barron.png)|![CVaR](img/basics_colorbar_cvar.png)|![Tilted](img/basics_colorbar_entropic.png)|![DRO](img/basics_colorbar_dro.png)|

Starting from the left-most figure, we have the graph of $x \mapsto \rho_{\alpha}(x/\sigma)$ plotted for numerous values of $\alpha$, with $\sigma = 0.2$ for ease of visual comparison with other functions. Next we have the graphs of $\phi(x)$ for both CVaR and tilted risk over different choices of $\beta$ and $\gamma$. Finally, for $\chi^{2}$-DRO risk we have plotted the analogous function $\phi(x) = (1+2a)(x)_{+}^{2} - x$, where $a$ is re-parametrized using $\widetilde{a}$ as before. The asymmetry and fast growth of the OCE and $\chi^{2}$-DRO risks is obvious, and is in stark contrast with both the symmetry and potentially sub-linear growth of the Barron class.

With the basic features of this function $\rho_{\alpha}$ understood, the general purpose risk class studied in \[Hol22\] is defined as

\begin{align}
\mathrm{R}_{\text{T}}(h;\theta) = \eta\theta + \mathbf{E}_{\mu}\rho_{\alpha}\left(\frac{\mathrm{L}(h)-\theta}{\sigma}\right).
\end{align}

This is called the threshold risk (or T-risk), where $\theta$ is the "threshold" about which the function $\rho_{\alpha}$ is used to measure dispersion, after re-scaling with $\sigma$. Note that the threshold is left free, but as a natural special case, the minimal T-risk

\begin{align}
\mathrm{R}_{\text{T}}(h) = \inf_{\theta \in \mathbb{R}} \mathrm{R}_{\text{T}}(h;\theta)
\end{align}

gives us a risk function whose form looks very similar to the OCE and $\chi^{2}$-DRO risks. Like these risks, the "location" captured by each risk is determined implicitly via the function measuring deviations, but in a bi-directional manner, and with more control over tail sensitivity than we get from just using the sum of mean and variance. On the computational side, much like OCE and DRO risks, we can solve for $(h,\theta)$ jointly in a straightforward way. The only caveat is that since $\rho_{\alpha}$ is not monotonic, even if the base loss function is convex, the map $h \mapsto \mathrm{R}_{\text{T}}(h;\theta)$ need not be convex.

Let us proceed by once again visualizing a subset of the T-risk class. Fpr ease of comparison with the other risk classes, we will consider the "minimal" T-risk $\mathrm{R}_{\text{T}}(h) = \inf_{\theta} \mathrm{R}_{\text{T}}(h;\theta)$. This means we can only take $\alpha \geq 1$ (otherwise $\mathrm{R}_{\text{T}}(h) = -\infty$). Since there are three influential parameters here ($\alpha$, $\sigma$, and $\eta$), we will have to take up each of them separately (with the others fixed). The following figure is analogous to the previous risk class visualizations done in this section.

![Risk visuals (T-risk, Bernoulli)](img/basics_riskvis_trisks_bernoulli.png)

Adjusting the $\alpha$ value has an effect similar to that seen in the M-location example much earlier. On the other hand, $\sigma$ and $\eta$ have a very different and quite pronounced effect. When $\eta \to 0$, the minimal T-risk value is precisely the generalized deviation incurred at the M-location induced by $\rho_{\alpha}$. When $\eta$ is negative, it shifts the optimal treshold upward, and when $\eta$ is positive, it pushes the optimal threshold downward. The scaling parameter $\sigma$ essentially controls the scale at which deviations are processed in a quadratic fashion versus a (potentially) linear fashion. For reference, let us visualize the minimal T-risk under the other three distributions examined previously.

![Risk visuals (T-risk, Beta)](img/basics_riskvis_trisks_beta.png)
![Risk visuals (T-risk, Pareto)](img/basics_riskvis_trisks_pareto.png)
![Risk visuals (T-risk, Normal)](img/basics_riskvis_trisks_normal.png)

A more detailed study of the key traits of the T-risk class is in the previously cited paper \[Hol22\].

## Loss transformations

Put roughly, the ultimate objective of machine learning is "good performance at test time given limited information at training time," and in this section we have been using the notion of a risk function $\mathrm{L} \mapsto \mathrm{R}(\mathrm{L})$ to explicitly define "good performance." Designing a risk function is great, but in the end, what we want is a learning algorithm that achieves small risk with sufficiently high confidence. How do we link up risk function design to learning procedures?

If we take $\mathrm{L}$ to be a test loss, the risk value $\mathrm{R}(\mathrm{L})$ is an ideal quantity, depending on knowledge of the true distribution of $\mathrm{L}$, which we of course do not have at training time. This means that if we want to find some $h \in \mathcal{H}$ such that $\mathrm{R}(h)$ is small, we will need come up with some data-driven estimates of $\mathrm{R}(\cdot)$ or some properties of $\mathrm{R}(\cdot)$ (e.g., gradients, etc.). Assuming still that $\mathrm{L}(h) = \ell(h;Z)$ for an arbitrary base loss function and $Z \sim \mu$, the next best thing to knowing $\mu$ is having an iid sample $Z_{1},\ldots,Z_{n}$ from $\mu$ available at training time. Arguably the simplest algorithmic strategy is as follows: _"replace $\mu$ with the empirical distribution, and use the resulting empirical risk as an objective function._ Let us see where this simple strategy leads us.

In the traditional setting of $\mathrm{R}(h) = \mathbf{E}_{\mu}\mathrm{L}(h)$, the resulting empirical objective is of course

\begin{align}
h \mapsto \frac{1}{n}\sum_{i=1}^{n}\mathrm{L}_{i}(h)
\end{align}

where we have denoted the sample losses by $\mathrm{L}_{i}(h) = \ell(h;Z_{i})$. It goes without saying that this is empirical risk minimization (ERM), the central paradigm of traditional statistical learning theory. Similarly, for the OCE risk we have

\begin{align}
(h,\theta) \mapsto \theta + \frac{1}{n}\sum_{i=1}^{n}\phi(\mathrm{L}_{i}(h)-\theta).
\end{align}

For the $\chi^{2}$-DRO risk we have

\begin{align}
(h,\theta) \mapsto \theta + \sqrt{(1+2a)\frac{1}{n}\sum_{i=1}^{n}(\mathrm{L}_{i}(h)-\theta)_{+}^{2}},
\end{align}

and for the T-risk we have

\begin{align}
(h,\theta) \mapsto \eta\theta + \frac{1}{n}\sum_{i=1}^{n}\rho_{\alpha}\left(\frac{\mathrm{L}_{i}(h)-\theta}{\sigma}\right).
\end{align}

Note that inherent in each of these estimators is a _transformation of the base loss function_. These transformations are as follows:

- ERM: $h \mapsto \ell(h;Z_{i})$ (trivial, no change)
- OCE: $h \mapsto \phi(\ell(h;Z_{i})-\theta)$
- $\chi^{2}$-DRO: $h \mapsto (1+2a)(\ell(h;Z_{i})-\theta)_{+}^{2}$
- T-risk: $h \mapsto \rho_{\alpha}(\ell(h;Z_{i})-\theta)$

One thus might be tempted to say that the design of new risk functions is just the design of new loss functions, and that we are still just doing ERM. Strictly speaking, of course one can take this viewpoint, but methodologically it seems wiser to separate _loss function design_ and _risk function design_ into two distinct elements of the machine learning workflow. We will discuss this more in later sections, but the most obvious reason is that loss functions are very tightly interwoven with each specific learning task we have. In particular, practitioners in computer vision and natural language processing fields have come up with all sorts of interesting and effective task-specific loss functions. On the other hand, the notion of "achieving a desirable loss distribution" is the explicit purpose of risk function design, and can be applied regardless of how the losses are computed.

To close this section, let us consider how the preceding "loss transformations" actually look when we plug in some typical concrete forms for $\ell(\cdot;\cdot)$. To start, let us assume the simplest possible case, where $h$ is just a real value we need to select, and let us start with a simple squared error function, i.e., $\ell(h) = (h-h^{\ast})^{2}$. We compute the transformed losses mentioned above for the T-risk, CVaR, tilted, and $\chi^{2}$-DRO risk classes below, with $h^{\ast} = \pi$.

![Loss transformations (squared)](img/basics_losstrans_squared.png)

The different colours used in each plot (reading left to right) correspond to different choices of $\alpha$, $\beta$, $\gamma$, and $\widetilde{a}$ respectively, taken over the same ranges as shown in the previous colourbars (noting however that there is no re-scaling with $\sigma$ here for T-risk). While it is obvious that the T-risk does not preserve convexity, for large enough values of $\alpha$, there is a form of weak convexity \[Hol22\]. Let us do a similar visualization for three other concrete loss functions, namely the absolute error $\ell(h) = \lvert h-h^{\ast} \rvert$, the hinge loss $\ell(h) = \max\{1-hh^{\ast},0\}$, and the cross-entropy loss $\ell(h) = \log(1+\exp(-hh^{\ast}))$.

![Loss transformations (squared)](img/basics_losstrans_absolute.png)
![Loss transformations (squared)](img/basics_losstrans_hinge.png)
![Loss transformations (squared)](img/basics_losstrans_cross-entropy.png)

Once again, we see how both the shape of the base loss function and the nature of the transformation are reflected quite clearly even in these simple examples. Now that we have some concrete ideas of how to transform losses into useful feedback given a wide range of risk functions, we will proceed in the following sections to actually look at real models, algorithms, and datasets.

___

## References

- \[Bar19\] Barron, J. T. (2019). A general and adaptive robust loss function. In Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition, pages 4331-4339.
- \[BJL15\] Brownlees, C., Joly, E., and Lugosi, G. (2015). Empirical risk minimization for heavy-tailed losses. The Annals of Statistics, 43(6):2507-2536.
- \[Hol21\] Holland, M. J. (2021) Learning with risks based on M-location. Machine Learning, to appear.
- \[Hol22\] Holland, M. J. (2022) Risk regularization through bidirectional dispersion. arXiv preprint arXiv:2203.14434.
- \[Kol97\] Koltchinskii, V. I. (1997). M-estimation, convexity and quantiles. The Annals of Statistics, pages 435-477.
- \[LBSS21\] Li, T., Beirami, A., Sanjabi, M., and Smith, V. (2021). Tilted empirical risk minimization. In The 9th International Conference on Learning Representations (ICLR).
- \[LPS20\] Lee, J., Park, S., and Shin, J. (2020). Learning bounds for risk-sensitive learning. In Advances in Neural Information Processing Systems 33 (NeurIPS 2020), pages 13867-13879.
- \[RU00\] Rockafellar, R. T. and Uryasev, S. (2000). Optimization of conditional value-at-risk. Journal of Risk, 2:21-42.
- \[ZDKR21\] Zhai, R., Dan, C., Kolter, J. Z., and Ravikumar, P. (2021). DORO: Distributional and outlier robust optimization. In 38th International Conference on Machine Learning (ICML), volume 139 of Proceedings of Machine Learning Research, pages 12345-12355.



___